<a href="https://colab.research.google.com/github/GOVINDFROMINDIA/IBM-watsonx/blob/main/watsonx_ai_summarize_product_reviews_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to analyze customer review comments to gain insights inorder to determine improvements in product

**Note:** Please note that for the watsonx challenge, please run these notebooks in IBM Cloud and not on on your laptop/desktop.

This notebook contains the steps and code to demonstrate support of text summarization in Watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

<a id="setup"></a>
## Set up the environment

### Install and import the dependecies

In [1]:
!pip install datasets | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install ibm-watson-machine-learning==1.0.349 | tail -n 1
!pip install rouge_score
!pip install evaluate

In [2]:
# Clone the repository
!git clone https://github.com/GOVINDFROMINDIA/IBM-watsonx.git

# Navigate into the directory
%cd IBM-watsonx


fatal: destination path 'IBM-watsonx' already exists and is not an empty directory.
/content/IBM-watsonx


In [3]:
import os, getpass
from pandas import read_csv
import evaluate

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. Instructions have been provided to generate IBM Cloud API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [4]:
!pip install ibm-cloud-sdk-core
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 13.8 MB/s eta 0:00:00
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.20.1-py3-none-any.whl size=101529 sha256=4decc0347e290f7e8b1ee5aa45d1047543c8d8bcccd69ed84e2e315e70006119
  Stored in directory: /root/.cache/pip/wheels/b1/80/5b/12af32c69e0bde3322cb9c1809952e8ed9f2a2a39509ee5374
Successfully built ibm-cloud-sdk-core
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: PyJWT
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3

Please enter your api key (hit enter): ··········


### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. When you run notebook on IBM Cloud, project in which it runs is saved as environment variable PROJECT_ID.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [5]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = "e7b00d41-c350-4245-b52d-fd5666f64c79"

<a id="data"></a>
## Train/test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [7]:
filename_test = '/content/IBM-watsonx/watsonx-ai-summarization-track-files/test.csv'
filename_train = '/content/IBM-watsonx/watsonx-ai-summarization-track-files/train.csv'

test_data = read_csv(filename_test)
train_data = read_csv(filename_train)

In [8]:
train_data.head()

,reviews,summary
0,"1: Very nice freezer works perfect, i'm so hap...",Features customers liked about the product: 1....
1,1: These machines are beautiful I chose the co...,Features customers liked about the product: 1....
2,"1: I am very happy with the way it cleans, and...",Features customers are happy about: 1. Cleanin...
3,"1: So happy with my new ABC stove, it’s super ...",Features customers liked about the ABC stove: ...
4,1: It really performs great and it is very eas...,Features that customers liked about the washer...


In [9]:
test_data.head()

,reviews,summary
0,1: (This review was collected as part of a pro...,Features customers liked about the washer: 1. ...
1,"1: Works great! No leaks, sturdy build, relati...",Features customers liked about the washer: 1. ...
2,"1: I’ve had it two months,very pleased quite,e...",Features customers liked about the washer: 1. ...
3,1: Fridge looks great and is working great\n2:...,Features customers liked about the product: 1....
4,1: I love the new washer works excellent\n2: I...,Features customers liked about the washer: 1. ...


<a id="models"></a>
## Foundation Models on Watsonx



Below code invokes Watson Machine Learning API to invoke Watsonx.ai LLMs


In [10]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"]
        else:
            return response.text

<a id="predict"></a>
## Evaluate the model, prompt and parameters

### **Analyze Product Reviews**

Define instructions for the model to summarize product reviews.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and tags like [Document], [End] and <|assistant|> which help to organize inputs and outputs in a better way. (reference - https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models-ibm-chat.html?context=wx)

It will act as a starting point for the participants. You will enhance it to get expected output.

Change the prompt or add your own examples or more examples and replance it in the below cell.


**Note:** In below prompt {reviews} needs to be replaced with input product review that needs to be summarised. Rest all is part of the prompt.

In [109]:
summarize_instruction = """

Analyze the given customer reviews and create two lists summarizing the product features:

1. Features customers liked about the product:
- List 8-12 positive features mentioned frequently in the reviews.
- Use simple, clear language matching the reviews closely.
- Start each point with an action verb or descriptive adjective when possible.
- Include specific product details, measurements, or comparisons if mentioned.
- Number each feature.

2. Features customers did not like about the product:
- If negative aspects are mentioned, list 1-3 issues.
- If no negatives are found, write exactly: "1. No specific features were mentioned as not being liked by customers in the given reviews."

Format your response exactly as follows:

Features customers liked about the product: 1. [Feature 1] 2. [Feature 2] 3. [Feature 3] 4. [Feature 4] 5. [Feature 5] 6. [Feature 6] 7. [Feature 7] 8. [Feature 8] [continue if more]

Features customers did not like about the product: 1. [Issue 1] 2. [Issue 2] (if applicable) 3. [Issue 3] (if applicable)

Critical instructions:
- Use only information directly stated in the reviews.
- Do not add any personal opinions or inferences.
- Keep language extremely simple and concise.
- Use exact headings as shown above, followed by a colon and space.
- Separate the two lists with a single blank line.
- For positive features, focus on performance, ease of use, design, and efficiency.
- For negative features, focus on any consistent complaints or issues mentioned.
[Document]
{reviews}
[End]


<|assistant|>

"""


### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model. parameters can be updated based on prompt.

In [110]:
parameters = {
    "decoding_method": "greedy",""
    "max_new_tokens": 300,
    "min_new_tokens": 50,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-chat-v2"

Analyze the summary of product reviews for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [111]:
results = []
prompt = Prompt(access_token, project_id)
product_reviews = list(test_data.reviews)
summary = list(test_data.summary.astype(str))

for review in product_reviews:
    # Below line of code replaces {reviews} from prompt with input product review that needs to be summarised. You might need to change or replace it based on your prompt to add review.
    prompt_instruction = summarize_instruction.format(reviews=review)
    results.append(prompt.generate(prompt_instruction, model_id, parameters).replace("\n",""))

### Calculate the Rouge Scores

In [112]:
# load the rouge score logic for summarization evaluation
# To adjust this to different task other than summarization import different evalualtion metric from HF
rouge_scorer_hf = evaluate.load('rouge')

In [113]:
# test score: 50% of the words from the reference summary occur in the generated summary - should be rouge1 score should be 0.5
rouge_scores = rouge_scorer_hf.compute(predictions = results,
                        references = summary)

In [114]:
print(rouge_scores['rouge1'])

0.5384949737541938
